In [111]:
# Import required libraries

import pandas as pd
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline


In [112]:
# Load the environment variables from the .env file

load_dotenv()


True

In [113]:
#Create list of tickers for the bond portfolio

tickers = ["BLV", "BND", "EDV", "VCLT", "VGLT"]


#Review bond portfolio tickers

tickers


['BLV', 'BND', 'EDV', 'VCLT', 'VGLT']

In [114]:
# Set the variables for the Alpaca API and secret keys

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")


# Create the Alpaca tradeapi.REST object

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
    

In [115]:
# Set timeframe to 1D 

timeframe = "1D"


# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day

start_date = pd.Timestamp("2020-01-02", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()


In [116]:
# Use the Alpaca get_barset function to get current closing prices the bond portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame


bonds_prices = alpaca.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000
).df


# Review the bond tickers Alpaca DataFrame

bonds_prices.head()


BLV                                         BND  \
                             open      high       low   close   volume   open   
time                                                                            
2020-01-02 00:00:00-05:00  100.67  101.2500  100.6700  100.87  2428578  83.98   
2020-01-03 00:00:00-05:00  101.28  102.0000  101.2300  101.99  1201205  84.18   
2020-01-06 00:00:00-05:00  102.10  102.3900  101.3000  101.43   687743  84.30   
2020-01-07 00:00:00-05:00  101.28  101.3900  100.9500  100.99   643372  84.20   
2020-01-08 00:00:00-05:00  101.09  101.3976  100.4304  100.63   571043  84.13   

                                                           ...    VCLT  \
                              high    low  close   volume  ...    open   
time                                                       ...           
2020-01-02 00:00:00-05:00  84.1100  83.93  83.98  4221018  ...  101.80   
2020-01-03 00:00:00-05:00  84.2900  84.10  84.26  5432336  ...  102.05   
2020-01-06 00:00:00-05:00  84.3099  84.11  84.16  3845484  ...  102.64   
2020-01-07 00:00:00-05:00  84.2000  84.07  84.07  3115250  ...  101.69   
2020-01-08 00:00:00-05:00  84.1971  83.90  83.97  3637969  ...  101.52   

                                                             VGLT          \
                              high     low   close  volume   open    high   
time                                                                        
2020-01-02 00:00:00-05:00  102.150  101.48  101.68  461013  83.75  84.310   
2020-01-03 00:00:00-05:00  102.455  101.97  102.40  537791  84.78  85.310   
2020-01-06 00:00:00-05:00  102.700  101.75  101.83  573593  85.65  85.650   
2020-01-07 00:00:00-05:00  101.820  101.32  101.32  175668  84.76  84.907   
2020-01-08 00:00:00-05:00  101.820  100.98  101.11  322433  84.61  84.808   

                                                       
                               low    close    volume  
time                                                   
2020-01-02 00:00:00-05:00  83.7500  84.0132  478428.0  
2020-01-03 00:00:00-05:00  84.5600  85.2600  268534.0  
2020-01-06 00:00:00-05:00  84.7229  84.8382  259873.0  
2020-01-07 00:00:00-05:00  84.4100  84.4400  288386.0  
2020-01-08 00:00:00-05:00  83.6200  83.8800  279854.0  

[5 rows x 25 columns]

In [117]:
# Configure the Monte Carlo simulation to forecast 10 years cumulative returns
# The weights should be split evenly across bonds.
# Run 500 samples.

MC_bonds_10 = MCSimulation(
    portfolio_data = bonds_prices,
    weights = [.2, .2, .2, .2, .2],
    num_simulation = 1000,
    num_trading_days = 252*10)


# Review the simulation input data

MC_bonds_10.portfolio_data.head()

BLV                                       \
                             open      high       low   close   volume   
time                                                                     
2020-01-02 00:00:00-05:00  100.67  101.2500  100.6700  100.87  2428578   
2020-01-03 00:00:00-05:00  101.28  102.0000  101.2300  101.99  1201205   
2020-01-06 00:00:00-05:00  102.10  102.3900  101.3000  101.43   687743   
2020-01-07 00:00:00-05:00  101.28  101.3900  100.9500  100.99   643372   
2020-01-08 00:00:00-05:00  101.09  101.3976  100.4304  100.63   571043   

                                          BND                         ...  \
                          daily_return   open     high    low  close  ...   
time                                                                  ...   
2020-01-02 00:00:00-05:00          NaN  83.98  84.1100  83.93  83.98  ...   
2020-01-03 00:00:00-05:00     0.011103  84.18  84.2900  84.10  84.26  ...   
2020-01-06 00:00:00-05:00    -0.005491  84.30  84.3099  84.11  84.16  ...   
2020-01-07 00:00:00-05:00    -0.004338  84.20  84.2000  84.07  84.07  ...   
2020-01-08 00:00:00-05:00    -0.003565  84.13  84.1971  83.90  83.97  ...   

                             VCLT                                VGLT          \
                              low   close  volume daily_return   open    high   
time                                                                            
2020-01-02 00:00:00-05:00  101.48  101.68  461013          NaN  83.75  84.310   
2020-01-03 00:00:00-05:00  101.97  102.40  537791     0.007081  84.78  85.310   
2020-01-06 00:00:00-05:00  101.75  101.83  573593    -0.005566  85.65  85.650   
2020-01-07 00:00:00-05:00  101.32  101.32  175668    -0.005008  84.76  84.907   
2020-01-08 00:00:00-05:00  100.98  101.11  322433    -0.002073  84.61  84.808   

                                                                    
                               low    close    volume daily_return  
time                                                                
2020-01-02 00:00:00-05:00  83.7500  84.0132  478428.0          NaN  
2020-01-03 00:00:00-05:00  84.5600  85.2600  268534.0     0.014841  
2020-01-06 00:00:00-05:00  84.7229  84.8382  259873.0    -0.004947  
2020-01-07 00:00:00-05:00  84.4100  84.4400  288386.0    -0.004694  
2020-01-08 00:00:00-05:00  83.6200  83.8800  279854.0    -0.006632  

[5 rows x 30 columns]

In [118]:
# Run the Monte Carlo simulation to forecast 10 years cumulative returns

MC_bonds_10.calc_cumulative_return()


Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.991104,0.991359,0.995559,1.003807,0.992873,0.996848,0.989240,0.997164,1.000319,1.001964,...,0.996368,1.015185,0.999893,1.008318,1.004412,1.014557,1.003930,0.998308,1.003790,0.989892
2,0.986875,0.991057,0.991814,1.004049,0.995375,0.999890,0.989671,0.998766,0.998954,1.003921,...,0.990587,1.014801,1.005898,1.013670,0.986283,1.020938,1.013238,1.003968,1.013780,0.995731
3,0.986765,0.986090,0.992002,0.996985,0.988385,0.993912,0.980306,0.993004,1.000424,1.014873,...,0.989813,1.019848,1.006853,1.008775,0.991479,1.018712,1.009018,1.006484,1.009743,1.001534
4,0.991603,0.984824,0.997897,1.003216,0.985933,0.992469,0.978006,0.990085,0.999225,1.013833,...,0.984297,1.023530,1.008147,0.999411,0.994735,1.021690,1.019648,0.996064,1.015725,1.006778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2516,3.299253,2.330147,3.665205,2.331065,3.393945,2.987527,3.374632,4.664980,4.545729,5.856981,...,2.481426,3.272001,4.097261,3.660371,3.174818,2.741344,3.664639,2.746025,3.493391,3.516700
2517,3.272579,2.317187,3.708583,2.325682,3.396061,2.976558,3.391179,4.665098,4.543617,5.918495,...,2.471348,3.233171,4.101745,3.671806,3.169075,2.783060,3.707805,2.712682,3.483217,3.552192
2518,3.294073,2.327470,3.694779,2.316787,3.421839,2.959115,3.415101,4.703835,4.525228,5.914070,...,2.439036,3.230833,4.130230,3.671826,3.196080,2.805097,3.690098,2.700068,3.486436,3.548103
2519,3.279944,2.323381,3.665807,2.302475,3.418865,2.983225,3.436208,4.725977,4.519928,5.937155,...,2.427710,3.239863,4.130223,3.671472,3.187716,2.808552,3.695727,2.714050,3.471843,3.554516


In [119]:

# Create an empty DataFrame to populate the closing prices on the bonds

bonds_closing_prices = pd.DataFrame()


# Use for-loop to pull the tickers' closing prices and populate the bonds_closing_prices DataFrame

for ticker in tickers:
    bonds_closing_prices[ticker] = bonds_prices[ticker]["close"] 


# Review DataFrame

bonds_closing_prices


,BLV,BND,EDV,VCLT,VGLT
time,,,,,
2020-01-02 00:00:00-05:00,100.87,83.980,131.69,101.68,84.0132
2020-01-03 00:00:00-05:00,101.99,84.260,134.27,102.40,85.2600
2020-01-06 00:00:00-05:00,101.43,84.160,133.56,101.83,84.8382
2020-01-07 00:00:00-05:00,100.99,84.070,132.65,101.32,84.4400
2020-01-08 00:00:00-05:00,100.63,83.970,131.63,101.11,83.8800
...,...,...,...,...,...
2020-12-24 00:00:00-05:00,109.30,88.005,151.81,110.71,95.5900
2020-12-28 00:00:00-05:00,109.37,87.980,152.00,110.76,95.6200
2020-12-29 00:00:00-05:00,109.50,88.020,151.81,110.77,95.5100


In [120]:
# Calculate the bonds portfolio aggregate daily returns

bonds_portfolio_returns = bonds_closing_prices.sum(axis=1).pct_change().dropna()


# Review the first 5 rows of the aggregate daily returns

bonds_portfolio_returns.head()

time
2020-01-03 00:00:00-05:00    0.011841
2020-01-06 00:00:00-05:00   -0.004648
2020-01-07 00:00:00-05:00   -0.004642
2020-01-08 00:00:00-05:00   -0.004469
2020-01-09 00:00:00-05:00    0.003701
dtype: float64

In [121]:
# Calculate the bonds portfolio standard deviation

bonds_portfolio_std = bonds_portfolio_returns.std()


# Review the standard devations

bonds_portfolio_std


0.020902525838938424